# Make figures for the GEOS 505 Climate Dashboard

Rainey Aberle, Front End Team

Fall 2022

In [1]:
import pandas as pd
import numpy as np
import os
import panel as pn
pn.extension()
import xarray as xr
import folium as fm
import matplotlib
import matplotlib.pyplot as plt
import hvplot.pandas
import holoviews as hv
import branca
import glob
import io
from PIL import Image

## Define paths in directory, load data

In [2]:
# -----Path to data
data_path = '/Users/raineyaberle/Courses/GEOS_505_ResearchComputing/data/dashboard/'

# -----Path where outputs will be saved
out_path = data_path + '../../dashboard/GEOS505-front-end-team/'

# -----Load data files as xarray.Datasets
os.chdir(data_path)
# precipitation rate
ds_p = xr.open_dataset('cfs_prate_20221130.nc')
# snow depth
ds_sde = xr.open_dataset('cfs_sde_20221130.nc')
# air temperature
ds_t = xr.open_dataset('cfs_t_20221130.nc')
# water runoff
ds_wr = xr.open_dataset('cfs_watr_20221130.nc')

# -----Merge Datasets
ds = xr.merge([ds_p, ds_sde, ds_t, ds_wr])
# replace no data values with NaN
# ds = ds.where(ds!=3.4028234663852886e+38)
ds

<xarray.Dataset>
Dimensions:     (valid_time: 28, latitude: 190, longitude: 384)
Coordinates:
    time        datetime64[ns] 2022-11-30
    step        (valid_time) timedelta64[ns] 0 days 00:00:00 ... 6 days 18:00:00
    surface     float64 0.0
  * latitude    (latitude) float64 89.28 88.34 87.4 ... -87.4 -88.34 -89.28
  * longitude   (longitude) float64 0.0 0.9375 1.875 2.812 ... 357.2 358.1 359.1
  * valid_time  (valid_time) datetime64[ns] 2022-11-30 ... 2022-12-06T18:00:00
Data variables:
    prate       (valid_time, latitude, longitude) float32 ...
    sde         (valid_time, latitude, longitude) float32 ...
    t           (valid_time, latitude, longitude) float32 ...
    watr        (valid_time, latitude, longitude) float32 ...

### Subset the data to the PNW

In [3]:
# grab a spatial subset for the first time slice
ds_PNW = ds.where((ds.latitude > 35) & (ds.latitude < 50)
                  & (ds.longitude > 230) & (ds.longitude < 250), drop=True)
ds_PNW

<xarray.Dataset>
Dimensions:     (valid_time: 28, latitude: 16, longitude: 21)
Coordinates:
    time        datetime64[ns] 2022-11-30
    step        (valid_time) timedelta64[ns] 0 days 00:00:00 ... 6 days 18:00:00
    surface     float64 0.0
  * latitude    (latitude) float64 49.61 48.66 47.72 46.77 ... 37.32 36.38 35.43
  * longitude   (longitude) float64 230.6 231.6 232.5 ... 247.5 248.4 249.4
  * valid_time  (valid_time) datetime64[ns] 2022-11-30 ... 2022-12-06T18:00:00
Data variables:
    prate       (valid_time, latitude, longitude) float32 0.000109 ... 1.2e-05
    sde         (valid_time, latitude, longitude) float32 nan nan ... 0.0 0.0
    t           (valid_time, latitude, longitude) float32 283.7 283.1 ... 279.5
    watr        (valid_time, latitude, longitude) float32 nan nan ... 0.08

### Create interactive map and scatterplot for each of the data variables

In [4]:
# -----Create drop down bar to select forecast timeframe
select = pn.widgets.Select(name='Select forecast timeframe:', options=['Day', 'Week', 'Month'])
select

Select(options=['Day', 'Week', 'Month'], value='Day')

In [7]:
# -----Set up for plotting
### Identify start and end times
# for the sample dataset, we'll set today as the first valid_time in the dataset
# t_start = np.datetime64('today')
t_start = ds['valid_time'].data[0]
if select.value=='Day':
    t_end = t_start + np.timedelta64(1, 'D')
elif select.value=='Week':
    t_end = t_start + np.timedelta64(7, 'D')
elif select.value=='Month':
    t_end = t_start + np.timedelta64(30, 'D')
### Calculate median of each variable over time (for map)
ds_PNW_time_median = ds_PNW.sel(valid_time=slice(t_start, t_end)).median(dim='valid_time')
### Calculate median and standard deviation of each variable over space (for line chart)
ds_PNW_space_median = ds_PNW.sel(valid_time=slice(t_start, t_end)).median(dim=['latitude', 'longitude'])
ds_PNW_space_std = ds_PNW.sel(valid_time=slice(t_start, t_end)).std(dim=['latitude', 'longitude'])
### Plot settings
# data variables to plot
data_vars = ['prate', 'sde', 't', 'watr']
# display names for each data variable
data_vars_display = ['Preciptiation rate [kg m^-2 s^-1]', 
                     'Snow depth [m]', 
                     'Air temperature [K]', 
                     'Water runoff [kg m^-2]']
# colormaps for map
cmaps = [matplotlib.cm.get_cmap('Blues'), 
         matplotlib.cm.get_cmap('cool'),
         matplotlib.cm.get_cmap('coolwarm'),
         matplotlib.cm.get_cmap('GnBu')]
# colors for line charts
chart_colors = [cmaps[0](150),
                cmaps[1](50),
                cmaps[2](150),
                cmaps[3](150)
               ]
# min and max of map
xmin, xmax = np.min(ds_PNW.longitude.data), np.max(ds_PNW.longitude.data)
ymin, ymax = np.min(ds_PNW.latitude.data), np.max(ds_PNW.latitude.data)
# function to "colorize" the data
# from: https://www.linkedin.com/pulse/visualize-dem-interactive-map-chonghua-yin/?trk=related_artice_Visualize%20DEM%20in%20An%20Interactive%20Map_article-card_title
def colorize(array, cmap):
    normed_data = (array - np.nanmin(array)) / (np.nanmax(array) - np.nanmin(array))   
    cm = cmap
    return cm(normed_data)

# -----Loop through data variables
for i, data_var in enumerate(data_vars):
    
    print(data_var)
    print('---------')
    
    # -----Create line chart
    # create pandas.DataFrame
    df = pd.DataFrame()
    df['valid_time'] = ds_PNW_space_median.valid_time.data
    df['median'] = ds_PNW_space_median['prate'].data
    df['low'] = ds_PNW_space_median['prate'].data - ds_PNW_space_std['prate'].data
    df['high'] = ds_PNW_space_median['prate'].data + ds_PNW_space_std['prate'].data
    # line plot for median, area plot for std
    std_plot = df.hvplot.area(x='valid_time', y='low', y2='high', label='standard deviation', color=chart_colors[i])
    med_plot = df.hvplot.line(x='valid_time', y='median', label='median', color='black')
    list_of_curves = [std_plot, med_plot]
    # add list of curves to plot
    chart = hv.Overlay(list_of_curves).opts(
        height=500, 
        width=800,
        ylabel=data_vars_display[i],
        xlabel='',
        title='Median and standard deviation of ' + data_var + ' for the next ' + str(select.value),
        legend_position='bottom_right',
    )
    # save chart to file
    fn = out_path + 'chart_'+data_var+'.html'
    hvplot.save(chart, fn)
    print('chart saved to file: '+fn)
    
    # -----Create map
    m = fm.Map(location=[np.nanmean(ds_PNW.latitude.data),  # mean latitude value in data 
                         np.nanmean(ds_PNW.longitude.data)], # mean longitude value in data
                      zoom_start=4, # initial map zoom level
                      tiles='StamenTerrain', # basemap
                      width=500, # map width
                      height=400) # map height
    # create colormap for legend
    cmap_legend = branca.colormap.LinearColormap([cmaps[i](j) for j in np.arange(0,256)], 
                                                 vmin=np.nanmin(ds_PNW_time_median[data_var]), 
                                                 vmax=np.nanmax(ds_PNW_time_median[data_var]), 
                                                 caption=data_vars_display[i], 
                                                 tick_labels=[np.nanmin(ds_PNW_time_median[data_var].data),
                                                              np.nanmax(ds_PNW_time_median[data_var].data),
                                                              np.nanmax(ds_PNW_time_median[data_var].data)]
                                                )
    # colorize the data
    data_colorized = colorize(ds_PNW_time_median[data_var].data, cmaps[i])
    # add image to map
    fm.raster_layers.ImageOverlay(image=data_colorized, 
                                  bounds=[[ymin, xmin], [ymax, xmax]], 
                                  opacity=0.8,
                                  origin='upper', 
                                ).add_to(m)
    # add colormap legend to map
    m.add_child(cmap_legend)
    # save to file as png
    fn = out_path + 'map_'+data_var + ".html"
    m.save(fn)
    print('map saved to file: ' + fn)
    print(' ')


prate
---------
chart saved to file: /Users/raineyaberle/Courses/GEOS_505_ResearchComputing/data/dashboard/../../dashboard/GEOS505-front-end-team/chart_prate.html
map saved to file: /Users/raineyaberle/Courses/GEOS_505_ResearchComputing/data/dashboard/../../dashboard/GEOS505-front-end-team/map_prate.html
 
sde
---------
chart saved to file: /Users/raineyaberle/Courses/GEOS_505_ResearchComputing/data/dashboard/../../dashboard/GEOS505-front-end-team/chart_sde.html
map saved to file: /Users/raineyaberle/Courses/GEOS_505_ResearchComputing/data/dashboard/../../dashboard/GEOS505-front-end-team/map_sde.html
 
t
---------
chart saved to file: /Users/raineyaberle/Courses/GEOS_505_ResearchComputing/data/dashboard/../../dashboard/GEOS505-front-end-team/chart_t.html
map saved to file: /Users/raineyaberle/Courses/GEOS_505_ResearchComputing/data/dashboard/../../dashboard/GEOS505-front-end-team/map_t.html
 
watr
---------
chart saved to file: /Users/raineyaberle/Courses/GEOS_505_ResearchComputing/dat

__NOTE:__ The Folium maps and charts can be added to the Panel dashboard as in the following command (like [this example](https://panel.holoviz.org/gallery/external/Folium.html)):

`pn.panel(map, height=400)`